In [16]:
#Libraries
require(data.table, quietly = TRUE)
require(glue, quietly = TRUE)
require(ggplot2, quietly = TRUE)
require(scatterplot3d, quietly = TRUE)
require(gridExtra, quietly = TRUE)
require(tidyr, quietly = TRUE)
require(tseries, quietly = TRUE)
library(pROC, quietly = TRUE)
library(caret, quietly = TRUE)
library(rpart, quietly = TRUE)
library(rpart.plot, quietly = TRUE)
library(e1071, quietly = TRUE)

In [17]:
trainpath <- "Data/IE582_Fall20_ProjectTrain.csv"
traindata <- read.csv(trainpath)

submitdatapath <- "Data/IE582_Fall20_ProjectTest.csv"
submitdata <- read.csv(submitdatapath)

submitdata <- submitdata[,-61] # Empty y column of the submission data set

In [18]:
traindata = traindata[,-c(50,52)]
submitdata = submitdata[,-c(50,52)]

In [19]:
dim(traindata)
dim(submitdata)

[1] 2074   59

[1] 2073   58

In [20]:
factor2 <- c()
for(i in 1:58)
{
  if(nlevels(as.factor(traindata[,i]))==2)
  {
    factor2 <- c(factor2,i)
  }
}

In [21]:
train3 = (t(sapply(traindata[,factor2],table)))

In [22]:
train3

,0,1
x2,682,1392
x3,701,1373
x4,642,1432
x12,1363,711
x13,2005,69
x15,312,1762
x16,1840,234
x17,1579,495
x18,2032,42
x19,1982,92


In [23]:
cdeneme <- c()
for(i in 1:nrow(train3))
{
  if(train3[i,2]<50)
  {
    cdeneme <- c(cdeneme, i)
  }
}

In [24]:
train3[cdeneme,]

,0,1
x18,2032,42
x26,2057,17
x37,2073,1
x46,2059,15
x49,2053,21
x55,2038,36
x57,2072,2
x59,2061,13


In [25]:
train5 = traindata[,-c(18,26,37,46,49,53,55,57)]

In [26]:
testdata2 = submitdata[,-c(18,26,37,46,49,53,55,57)]

In [27]:
set.seed(1)
trainIndex = createDataPartition(train5$y, p = 0.7, list = FALSE)
traindata07 = train5[trainIndex, ]
traindata03 = train5[-trainIndex, ]

In [15]:
traindata07[traindata07$x42 >= 1,'x42'] = 1
traindata07[traindata07$x36 >= 1,'x36'] = 1

traindata03[traindata03$x42 >= 1,'x42'] = 1
traindata03[traindata03$x36 >= 1,'x36'] = 1

ERROR: Error in bigscaled[bigscaled$x42 >= 1, "x42"] = 1: 'bigscaled' nesnesi bulunamadı


In [13]:
fitControl <- trainControl(method = "repeatedcv", number = 10, repeats = 3, summaryFunction = twoClassSummary,
                           verboseIter = TRUE, classProbs = TRUE, sampling = "up")
tunegrid <- expand.grid(.mtry=c(3,5,10,15), .splitrule = "gini", .min.node.size = c(3,5,7))

In [14]:
RF20 <- train(y~., data = train5, method="ranger",  num.trees = 350, metric = "ROC", preProc=c("center", "scale"),
              trControl = fitControl, tuneGrid = tunegrid, importance = 'permutation')
RF20

+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=7 
- Fold01.Rep1: mtry= 3, splitru

Selecting tuning parameters
Fitting mtry = 5, splitrule = gini, min.node.size = 7 on full training set


Random Forest 

2074 samples
  50 predictor
   2 classes: 'a', 'b' 

Pre-processing: centered (50), scaled (50) 
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1866, 1866, 1867, 1866, 1867, 1867, ... 
Addtional sampling using up-sampling prior to pre-processing

Resampling results across tuning parameters:

  mtry  min.node.size  ROC        Sens       Spec     
   3    3              0.8804145  0.8632098  0.6915686
   3    5              0.8795641  0.8587335  0.6929281
   3    7              0.8803098  0.8536270  0.7014248
   5    3              0.8816475  0.8955958  0.6320392
   5    5              0.8821644  0.8951671  0.6398824
   5    7              0.8823605  0.8913387  0.6464444
  10    3              0.8808703  0.8983614  0.6195425
  10    5              0.8818630  0.8949494  0.6260915
  10    7              0.8810796  0.8970861  0.6418170
  15    3              0.8801227  0.9011174  0.6300523
  15    5              0.8788227  0.8987887  0.6307

In [15]:
predtest20 <- predict(RF20,testdata2,type='prob')  #testdata 
predtest20$b

[1] 4.542857e-02 1.835329e-01 3.911139e-01 7.602991e-02 6.704673e-01
   [6] 1.865695e-01 3.999781e-01 4.445202e-01 1.428503e-01 2.896825e-03
  [11] 3.159121e-01 3.561622e-01 3.212336e-02 8.606622e-01 2.031303e-01
  [16] 5.132820e-02 2.135374e-02 2.138154e-01 2.553041e-01 2.995673e-02
  [21] 1.190761e-01 4.051852e-01 8.893200e-01 3.968254e-05 6.668019e-01
  [26] 7.889453e-02 5.010851e-01 3.937778e-01 5.234905e-01 3.420751e-01
  [31] 4.890127e-01 7.544170e-01 1.662751e-02 3.925008e-01 6.111551e-01
  [36] 2.972085e-01 7.298765e-01 7.460437e-01 8.860626e-01 2.539116e-02
  [41] 2.256588e-01 1.877259e-01 1.132007e-01 7.229317e-01 2.493534e-01
  [46] 7.854302e-01 2.647680e-01 3.688854e-02 3.090522e-01 1.465216e-01
  [51] 2.287493e-02 3.310892e-02 1.883284e-01 3.367424e-01 8.218361e-01
  [56] 2.974150e-01 7.069804e-01 5.390533e-01 1.262015e-01 5.146868e-01
  [61] 8.324546e-01 2.494426e-02 7.032744e-02 1.856706e-01 7.428571e-02
  [66] 8.641497e-02 5.394338e-03 6.047619e-03 6.566968e-01 2.303498e-01
  [71] 6.015113e-01 4.106263e-02 4.614739e-02 6.806667e-01 9.905231e-02
  [76] 2.487520e-01 6.732572e-01 4.390083e-02 4.286198e-01 1.085034e-01
  [81] 8.884228e-01 1.187181e-01 1.508935e-02 5.298032e-01 2.765374e-01
  [86] 2.478676e-01 6.896912e-03 1.014492e-01 7.929738e-01 2.738109e-01
  [91] 3.587075e-02 3.238095e-02 1.066461e-01 7.429317e-01 1.397073e-01
  [96] 4.071693e-01 1.393148e-01 7.165015e-02 4.071578e-01 4.768144e-01
 [101] 6.402168e-01 3.004082e-02 6.022135e-01 3.930123e-01 4.019586e-01
 [106] 3.776872e-01 1.906806e-02 2.329615e-01 2.232785e-01 5.043197e-02
 [111] 4.229932e-02 8.180064e-01 1.386871e-01 6.915697e-01 2.145299e-01
 [116] 2.760012e-02 2.059955e-02 3.591984e-01 3.633333e-02 3.614267e-02
 [121] 3.581544e-02 7.011434e-02 5.731044e-02 1.747614e-01 2.294638e-01
 [126] 2.359104e-02 3.186905e-02 3.251988e-01 6.553660e-01 3.619048e-02
 [131] 6.373232e-01 4.261032e-01 3.174611e-01 3.669925e-01 4.869411e-01
 [136] 6.577488e-01 7.328663e-02 6.532187e-01 1.301173e-01 6.617068e-01
 [141] 3.764073e-01 2.196695e-01 2.901803e-01 5.238076e-01 4.187692e-01
 [146] 2.671798e-01 6.706840e-01 2.219575e-01 3.250760e-01 1.167700e-01
 [151] 9.260701e-02 3.444955e-01 2.547761e-01 1.610817e-01 2.105261e-02
 [156] 1.805154e-01 5.827683e-02 4.678050e-01 3.995839e-01 9.080860e-01
 [161] 1.552457e-01 1.966586e-01 3.101247e-02 7.414485e-02 4.969743e-02
 [166] 5.802921e-01 7.041534e-02 5.655672e-01 3.231293e-02 1.931685e-01
 [171] 7.517896e-01 3.062638e-01 3.967387e-01 4.047619e-02 4.137189e-01
 [176] 2.439456e-02 2.703079e-01 3.817886e-01 6.621684e-01 8.793572e-01
 [181] 7.926426e-01 7.275187e-01 4.789233e-01 7.741497e-03 8.318258e-01
 [186] 2.970068e-02 3.968254e-05 2.370191e-01 2.113357e-01 2.181281e-01
 [191] 7.531179e-03 4.472248e-01 6.591790e-01 2.062068e-01 1.820295e-02
 [196] 5.893720e-03 4.685714e-02 2.927751e-02 3.734448e-02 7.643769e-01
 [201] 5.316277e-01 1.897835e-01 1.875317e-01 5.902119e-01 4.068144e-01
 [206] 5.584554e-01 4.177890e-02 8.901072e-01 1.013231e-01 7.475152e-01
 [211] 6.480427e-02 3.998866e-03 6.008730e-02 4.793603e-01 1.616216e-01
 [216] 7.125561e-01 1.816045e-01 5.376190e-02 2.020408e-02 1.074085e-01
 [221] 1.114159e-01 2.531954e-02 3.651031e-01 6.098059e-01 4.368213e-02
 [226] 1.403844e-01 1.671335e-01 7.205652e-01 2.446985e-01 1.850397e-02
 [231] 8.785858e-01 6.185201e-02 4.208874e-01 5.769029e-02 1.184762e-02
 [236] 5.838805e-02 4.424667e-01 1.554591e-01 5.889651e-01 5.942445e-01
 [241] 5.911505e-02 2.203078e-02 5.391383e-02 7.326962e-02 7.179252e-02
 [246] 3.628073e-02 4.432381e-01 6.463609e-01 4.273520e-01 1.181271e-02
 [251] 5.068963e-02 7.124493e-01 7.406617e-01 3.994249e-01 6.950680e-02
 [256] 6.015270e-01 8.529778e-01 2.863383e-01 2.086939e-01 1.684420e-01
 [261] 5.491957e-01 2.546803e-01 8.558484e-02 2.832528e-01 7.015695e-01
 [266] 3.151036e-01 4.607256e-02 1.226443e-01 1.006495e-01 1.127789e-01
 [271] 3.160471e-01 1.917751e-01 2.494245e-01 2.706927e-01 6.971520e-02
 [276] 1.975407e-01 5.745808e-01 6.426745e-

In [18]:
predictions <- predtest20$b

In [19]:
send_submission(predictions, token, url=subm_url, submit_now= submit_now)

[1] "Format OK"
$submission
[0.0454,0.1835,0.3911,0.076,0.6705,0.1866,0.4,0.4445,0.1429,0.0029,0.3159,0.3562,0.0321,0.8607,0.2031,0.0513,0.0214,0.2138,0.2553,0.03,0.1191,0.4052,0.8893,0,0.6668,0.0789,0.5011,0.3938,0.5235,0.3421,0.489,0.7544,0.0166,0.3925,0.6112,0.2972,0.7299,0.746,0.8861,0.0254,0.2257,0.1877,0.1132,0.7229,0.2494,0.7854,0.2648,0.0369,0.3091,0.1465,0.0229,0.0331,0.1883,0.3367,0.8218,0.2974,0.707,0.5391,0.1262,0.5147,0.8325,0.0249,0.0703,0.1857,0.0743,0.0864,0.0054,0.006,0.6567,0.2303,0.6015,0.0411,0.0461,0.6807,0.0991,0.2488,0.6733,0.0439,0.4286,0.1085,0.8884,0.1187,0.0151,0.5298,0.2765,0.2479,0.0069,0.1014,0.793,0.2738,0.0359,0.0324,0.1066,0.7429,0.1397,0.4072,0.1393,0.0717,0.4072,0.4768,0.6402,0.03,0.6022,0.393,0.402,0.3777,0.0191,0.233,0.2233,0.0504,0.0423,0.818,0.1387,0.6916,0.2145,0.0276,0.0206,0.3592,0.0363,0.0361,0.0358,0.0701,0.0573,0.1748,0.2295,0.0236,0.0319,0.3252,0.6554,0.0362,0.6373,0.4261,0.3175,0.367,0.4869,0.6577,0.0733,0.6532,0.1301,0.6617,0.3764,0.2197,

In [50]:
set.seed(600)
RF10 <- train(y~., data=traindata07, method="ranger", trControl = fitControl, tuneGrid = tunegrid)

+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=3 
- Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=3 
+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=15, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 3, splitrule=gini, min.node.size=7 
- Fold01.Rep1: mtry= 3, splitru

Selecting tuning parameters
Fitting mtry = 5, splitrule = gini, min.node.size = 5 on full training set


In [51]:
RF10

Random Forest 

1453 samples
  50 predictor
   2 classes: 'a', 'b' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 5 times) 
Summary of sample sizes: 1308, 1307, 1307, 1308, 1307, 1309, ... 
Resampling results across tuning parameters:

  mtry  min.node.size  Accuracy   Kappa    
   3    3              0.8337333  0.4660086
   3    5              0.8342889  0.4677239
   3    7              0.8351127  0.4693805
   5    3              0.8353923  0.4929895
   5    5              0.8359441  0.4933707
   5    7              0.8352497  0.4914812
  10    3              0.8318099  0.4946252
  10    5              0.8289247  0.4878990
  10    7              0.8297456  0.4897051
  15    3              0.8296039  0.4943051
  15    5              0.8296124  0.4938558
  15    7              0.8302974  0.4973059

Tuning parameter 'splitrule' was held constant at a value of gini
Accuracy was used to select the optimal model using the largest value.
The final values used for the mode

In [52]:
predtest5 <- predict(RF10,traindata03)  #testdata 
predtest5

[1] a a a a a a a a a b a a a a a b a a a b a a b a a a a b a a b b a a a a a
 [38] a a b b a a b a a a a a a b a b a b a a a a a a a a a a a a b a a a a a a
 [75] a a a a a b a a a a a a a a a a a a a a a a a a a a a a a a a a a a a b a
[112] a a b a a a a a a a a a a a a b a a a a a a a a a a a a a a a a a a b a a
[149] a a a a a a a b a a a b a a b a a b a a a a b a a b a a a b a a a b b a a
[186] b a a a a b a b a a a a a a a a a a a a b b a a b a a a b a a a b b a a a
[223] a a a a a a a a b a b a a a b a a b a b a a a a a a a a a a a a a a a a b
[260] a b a a a a a a b b a b a a a a a a a a a a b a a b a a a a a b b b b a a
[297] a a a a a a a a a a a a a a a a a b a b a a a a a a a a a a a a b a a b a
[334] b b a a a a a b a a a a b b a b a b a b a b a a a a b a a b a a a b a b b
[371] a a b a b b b b a a a a a a b a a a a b a a a a a a a a a a a a a a a b a
[408] a a a a a b b a a b a a a a a a a a a a a a a a a a b a a a a a a a a a a
[445] a a a a a b b b a a a a a a b a a a b a b a a a a a a a a a a a a a a a a
[482] b a a a a a a a a a b a a a a a a a a a a a b a a a a a a a a a b a a a a
[519] a a a a a a b a a a a a a a a b a a a a a a b b a a b a a a a a a a a a b
[556] b a a b a b a b a a a a a a a a a a a a a a a a a a b b a a a a b a a a a
[593] a b a a a a a a a a a a a a a a a a b a a b a a b a a a a
Levels: a b

In [53]:
get_accuracy <- function(predicted, actual){
  confusion_table = table(predicted,actual)
  TP = confusion_table[2,2]
  TN = confusion_table[1,1]
  FN = confusion_table[1,2]
  FP = confusion_table[2,1]
  accuracy = ((FP/(sum(TN+FP))+FN/(sum(FN+TP)))*0.5)
  return(accuracy)
}

In [54]:
1-get_accuracy(predtest5,traindata03$y)

[1] 0.7355305

In [57]:
predtest5 <- predict(RF10,submitdata,type='prob')  #testdata 
predtest5$b

[1] 0.074729488 0.211556644 0.280949842 0.069588654 0.613259525 0.139729551
   [7] 0.336093132 0.291433271 0.054895880 0.005166913 0.439651486 0.296453646
  [13] 0.042036841 0.790454166 0.105464714 0.019300729 0.020179040 0.229893455
  [19] 0.244280765 0.020115889 0.097636918 0.301475215 0.827312403 0.018334569
  [25] 0.524387633 0.032153411 0.368039275 0.465030844 0.412215518 0.311308848
  [31] 0.347224807 0.649364371 0.018496238 0.272199455 0.548290696 0.309651603
  [37] 0.710463163 0.648702888 0.787385592 0.014758805 0.144972729 0.133077948
  [43] 0.047195985 0.601786789 0.264173099 0.630323221 0.226532676 0.007238405
  [49] 0.271686460 0.133799189 0.008324767 0.036675236 0.206502428 0.187658298
  [55] 0.776231638 0.388972549 0.527309116 0.398475378 0.108269477 0.289397617
  [61] 0.789352342 0.021902089 0.042573132 0.172890023 0.049964583 0.090811373
  [67] 0.006619417 0.028833333 0.619184180 0.130581827 0.372854750 0.024941855
  [73] 0.041248785 0.389919360 0.060887318 0.179088442 0.513889004 0.012940138
  [79] 0.209302137 0.129857277 0.793657619 0.129475971 0.016300630 0.543121348
  [85] 0.265527895 0.144960436 0.005547940 0.045693280 0.759650383 0.200720401
  [91] 0.011017156 0.094093256 0.066582868 0.685220307 0.120417281 0.364101760
  [97] 0.116516990 0.065484405 0.332830888 0.361276109 0.479353242 0.003660263
 [103] 0.537863159 0.226594889 0.328912528 0.281553809 0.029638514 0.187674454
 [109] 0.355851616 0.029820169 0.030953502 0.663685624 0.137159376 0.593236161
 [115] 0.288573299 0.028479638 0.026276477 0.277488700 0.035785555 0.040872160
 [121] 0.007992627 0.087350823 0.039040344 0.190293744 0.228215729 0.040883485
 [127] 0.061946609 0.323205565 0.568779217 0.075865965 0.534348952 0.184384718
 [133] 0.308255992 0.268263750 0.392952488 0.532305459 0.043144846 0.456379437
 [139] 0.037304958 0.492078385 0.351965817 0.167526466 0.229900133 0.415106548
 [145] 0.299782691 0.223033489 0.557313079 0.345593619 0.268146142 0.053469761
 [151] 0.100490738 0.196791917 0.229873450 0.165116889 0.025763370 0.088369798
 [157] 0.049228672 0.423901743 0.241482121 0.796293413 0.105280017 0.118335459
 [163] 0.034307643 0.096543084 0.035503738 0.365890962 0.089526743 0.485985955
 [169] 0.011466667 0.118451239 0.587565723 0.215823090 0.336237049 0.043792775
 [175] 0.288534994 0.025802499 0.281994936 0.357546979 0.623313276 0.791720114
 [181] 0.680228640 0.681875114 0.393140580 0.034393939 0.729450988 0.054456475
 [187] 0.004536084 0.130562503 0.232252869 0.139095025 0.012815920 0.354524239
 [193] 0.517518568 0.150168907 0.006794540 0.006361711 0.026990189 0.020112588
 [199] 0.035616667 0.574222132 0.464675686 0.162705924 0.206242310 0.408796240
 [205] 0.333295782 0.378035688 0.041434583 0.744358452 0.065050141 0.559586552
 [211] 0.031515448 0.008302005 0.043871476 0.483985859 0.264869306 0.540126437
 [217] 0.177601824 0.018959240 0.018461733 0.059113326 0.090531114 0.010038917
 [223] 0.307977694 0.559553436 0.049119263 0.100873135 0.105186970 0.560617334
 [229] 0.248994514 0.022431283 0.753675122 0.019045062 0.509307374 0.037574031
 [235] 0.017127961 0.028474131 0.399756409 0.294692627 0.243013050 0.634374766
 [241] 0.046166667 0.036965426 0.010361034 0.095928225 0.073205130 0.056041865
 [247] 0.457418562 0.437232660 0.445078876 0.015042424 0.061632672 0.774453586
 [253] 0.624621797 0.234780478 0.050605860 0.526919863 0.755753263 0.237973715
 [259] 0.039892402 0.227609929 0.473882490 0.212635247 0.033889024 0.300751262
 [265] 0.573047077 0.342353266 0.087571817 0.084851825 0.062826727 0.145832439
 [271] 0.245408912 0.144096895 0.190260059 0.339990265 0.032088140 0.191756738
 [277] 0.325760473 0.080860384 0.365092906 0.263535012 0.274732154 0.035195581
 [283] 0.266489322 0.080049488 0.503977236 0.260625539 0.073558698 0.453340891
 [289] 0.576282377 0.281709636 0.430061916 0.053587848 0.347169801 0.256426933
 [295] 0.322128044 0.781875166 0.577459520 0.073871727 0.050482224 0.080225270
 [301] 0.655139431 0.019661856 0.030369417 0.00869393

In [17]:
require(jsonlite)
require(httr)
require(data.table)
get_token <- function(username, password, url_site){
    
    post_body = list(username=username,password=password)
    post_url_string = paste0(url_site,'/token/')
    result = POST(post_url_string, body = post_body)

    # error handling (wrong credentials)
    if(result$status_code==400){
        print('Check your credentials')
        return(0)
    }
    else if (result$status_code==201){
        output = content(result)
        token = output$key
    }

    return(token)
}

send_submission <- function(predictions, token, url_site, submit_now=F){
    
    format_check=check_format(predictions)
    if(!format_check){
        return(FALSE)
    }
    
    post_string="list("
    for(i in 1:length(predictions)){
        if(i<length(predictions)){
            post_string=sprintf("%s%s,",post_string,predictions[i])
        } else {
            post_string=sprintf("%s%s)",post_string,predictions[i])
        }
    }
    
    submission = eval(parse(text=post_string))
    json_body = jsonlite::toJSON(submission, auto_unbox = TRUE)
    submission=list(submission=json_body)
    print(submission)

    if(!submit_now){
        print("You did not submit.")
        return(FALSE)      
    }
    

    header = add_headers(c(Authorization=paste('Token',token,sep=' ')))
    post_url_string = paste0(url_site,'/submission/')
    result = POST(post_url_string, header, body=submission)
    
    if (result$status_code==201){
        print("Successfully submitted. Below you can see the details of your submission")
    } else {
        print("Could not submit. Please check the error message below, contact the assistant if needed.")
    }
    
    print(content(result))
    
}

check_format <- function(predictions){
    
    if(all(is.numeric(predictions)) & all(predictions<=1)){
        print("Format OK")
        return(TRUE)
    } else {
        print("Wrong format")
        return(FALSE)
    }
    
}

# this part is main code
subm_url ='http://46.101.121.83'

u_name = "Los Galacticos"
p_word = "E6lOux9kirvumsWW"
submit_now = TRUE

username = u_name
password = p_word

token = get_token(username=u_name, password=p_word, url=subm_url)

Loading required package: jsonlite
Loading required package: httr

Attaching package: 'httr'

The following object is masked from 'package:caret':

    progress



In [60]:
predictions <- predtest5$b

In [61]:
send_submission(predictions, token, url=subm_url, submit_now= submit_now)

[1] "Format OK"
$submission
[0.0747,0.2116,0.2809,0.0696,0.6133,0.1397,0.3361,0.2914,0.0549,0.0052,0.4397,0.2965,0.042,0.7905,0.1055,0.0193,0.0202,0.2299,0.2443,0.0201,0.0976,0.3015,0.8273,0.0183,0.5244,0.0322,0.368,0.465,0.4122,0.3113,0.3472,0.6494,0.0185,0.2722,0.5483,0.3097,0.7105,0.6487,0.7874,0.0148,0.145,0.1331,0.0472,0.6018,0.2642,0.6303,0.2265,0.0072,0.2717,0.1338,0.0083,0.0367,0.2065,0.1877,0.7762,0.389,0.5273,0.3985,0.1083,0.2894,0.7894,0.0219,0.0426,0.1729,0.05,0.0908,0.0066,0.0288,0.6192,0.1306,0.3729,0.0249,0.0412,0.3899,0.0609,0.1791,0.5139,0.0129,0.2093,0.1299,0.7937,0.1295,0.0163,0.5431,0.2655,0.145,0.0055,0.0457,0.7597,0.2007,0.011,0.0941,0.0666,0.6852,0.1204,0.3641,0.1165,0.0655,0.3328,0.3613,0.4794,0.0037,0.5379,0.2266,0.3289,0.2816,0.0296,0.1877,0.3559,0.0298,0.031,0.6637,0.1372,0.5932,0.2886,0.0285,0.0263,0.2775,0.0358,0.0409,0.008,0.0874,0.039,0.1903,0.2282,0.0409,0.0619,0.3232,0.5688,0.0759,0.5343,0.1844,0.3083,0.2683,0.393,0.5323,0.0431,0.4564,0.0373,0.4921,0.35